# Cassandra Study 4-Data modeling 2

## Query-Driven Modeling Practice

In [1]:
import cassandra

In [2]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)    

In [3]:
try:
    query = "select * from hotels_by_poi"
    session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [4]:
try:
    session.execute("""
    create keyspace if not exists miketest
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [5]:
try:
    session.set_keyspace('miketest')
except Exception as e:
    print(e)        

### Defining application queries

Q1. Find hotels near a given point of interest.

Q2. Find information about a given hotel, such as its name and location.

Q3. Find points of interest near a given hotel.

Q4. Find an available room in a given date range.

Q5. Find the rate and amenities for a room.

Q6. Lookup a reservation by confirmation number.

Q7. Lookup all reservations by guest name.

Q8. Lookup a reservation by hotel, date, and guest name.

Q9. View guest details.

Reference
https://cassandra.apache.org/doc/latest/cassandra/data_modeling/data_modeling_queries.html

### Create Logical data model tables

In [43]:
query = "CREATE TABLE IF NOT EXISTS hotels_by_poi"
query = query + "(poi_name text, hotel_id int, name text, phone int, address text, PRIMARY KEY (poi_name, hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [44]:
query = "CREATE TABLE IF NOT EXISTS hotels"
query = query + "(hotel_id int, name text, phone text, address text, PRIMARY KEY (hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [45]:
query = "CREATE TABLE IF NOT EXISTS pois_by_hotel"
query = query + "(hotel_id int, poi_name text, description text, PRIMARY KEY (hotel_id, poi_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [46]:
query = "CREATE TABLE IF NOT EXISTS available_rooms_by_hotel_date"
query = query + "(hotel_id text, date date, room_number int, is_abailable boolean, PRIMARY KEY (hotel_id, date, room_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [47]:
query = "CREATE TABLE IF NOT EXISTS amenities_by_room"
query = query + "(hotel_id text, room_id int, amenity_name text, description text, PRIMARY KEY ((hotel_id, room_id), amenity_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [48]:
query = "CREATE TABLE IF NOT EXISTS reservations_by_confirmation"
query = query + "(confirm_number int, hotel_id int, room_id int, start_date date, end_date date, guest_id int, PRIMARY KEY (confirm_number, hotel_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [49]:
query = "CREATE TABLE IF NOT EXISTS reservations_by_guest"
query = query + "(guest_last_name text, hotel_id int, guest_id int, room_id int, start_date date, end_date date, confirm_number int, PRIMARY KEY (guest_last_name, hotel_id, guest_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [50]:
query = "CREATE TABLE IF NOT EXISTS reservations_by_hotel_date"
query = query + "(hotel_id int, start_date date, room_id int, end_date date, confirm_number int, gyest_id int, PRIMARY KEY ((hotel_id, start_date), room_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [51]:
query = "CREATE TABLE IF NOT EXISTS guests"
query = query + "(guest_id int, first_name text, last_name text, title text, email text, phone_numbers int, address text, PRIMARY KEY (guest_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### The above create table CQL is just for thinking, not the actual tables

In [57]:
query = "drop table hotels_by_poi"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table hotels"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table pois_by_hotel"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table available_rooms_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table amenities_by_room"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_confirmation"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_guest"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table guests"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

Error from server: code=2200 [Invalid query] message="Table 'miketest.hotels_by_poi' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.hotels' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.pois_by_hotel' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.available_rooms_by_hotel_date' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.amenities_by_room' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.reservations_by_confirmation' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.reservations_by_guest' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.reservations_by_hotel_date' doesn't exist"
Error from server: code=2200 [Invalid query] message="Table 'miketest.guests' doesn't exist"


### Transform into Physical Data Modeling

In [6]:
try:
    session.execute("""
    create keyspace if not exists hotel
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [32]:
try:
    session.set_keyspace('hotel')
except Exception as e:
    print(e)  

#### 1. create new column type about address data

In [34]:
query = "CREATE TYPE IF NOT EXISTS address "
query = query + "(street text, city text, state_or_province text, postal_code text, country text)"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### 2. Create hotels table include address type column.

In [35]:
query = "CREATE TABLE IF NOT EXISTS hotels "
query = query + "(id text, name text, phone text, address frozen<address>, PRIMARY KEY (id)) "
query = query + "WITH comment ='Q2. Find information about a hotel'"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### 3. Insert test row to check if the create table CQL currect and how it display data.

In [37]:
query = "INSERT INTO hotels (id, name, phone, address)"
query = query + " VALUES (%s, %s, %s, {street:%s, city:%s, state_or_province:%s, postal_code:%s, country:%s})"

try:
    session.execute(query, ("A1111", "Test Hotel", "062222222", "No. 1, Ln. 1, Daren St., West Central Dist.","Tainan","Taiwan","700004","Taiwan"))
except Exception as e:
    print(e)

In [39]:
query = 'SELECT * FROM hotels'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.id, row.name, row.phone, row.address)

A1111 Test Hotel 062222222 address(street='No. 1, Ln. 1, Daren St., West Central Dist.', city='Tainan', state_or_province='Taiwan', postal_code='700004', country='Taiwan')


#### 4. delete test row

In [40]:
query = "delete from hotels where id = 'A1111'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

#### 5. continue create other physical table.

In [41]:
query = "CREATE TABLE IF NOT EXISTS hotels_by_poi "
query = query + "(poi_name text, hotel_id text, name text, phone text, address frozen<address>, PRIMARY KEY (poi_name, hotel_id)) "
query = query + "WITH comment ='Q1. Find hotels near given poi' and CLUSTERING ORDER BY (hotel_id ASC)"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS pois_by_hotel"
query = query + "(hotel_id text, poi_name text, description text, PRIMARY KEY (hotel_id, poi_name))"
query = query + "WITH comment ='Q3. Find pois near a hotel'"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS available_rooms_by_hotel_date "
query = query + "(hotel_id text, date date, room_number smallint, is_abailable boolean, PRIMARY KEY (hotel_id, date, room_number))"
query = query + "WITH comment ='Q4. Find available rooms by hotel date'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "CREATE TABLE IF NOT EXISTS amenities_by_room "
query = query + "(hotel_id text, room_id smallint, amenity_name text, description text, PRIMARY KEY ((hotel_id, room_id), amenity_name)) "
query = query + "WITH comment ='Q5. Find amenities for a room'"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### If you need to drop table when you test, then below CQL can run

In [ ]:
query = "drop table hotels_by_poi"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table hotels"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table pois_by_hotel"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table available_rooms_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table amenities_by_room"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [42]:
try:
    session.execute("""
    create keyspace if not exists reservation
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }""")
except Exception as e:
    print(e)    

In [43]:
try:
    session.set_keyspace('reservation')
except Exception as e:
    print(e)  

In [45]:
query = "CREATE TYPE IF NOT EXISTS address "
query = query + "(street text, city text, state_or_province text, postal_code text, country text)"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [48]:
query = "CREATE TABLE IF NOT EXISTS reservations_by_confirmation "
query = query + "(confirm_number text, hotel_id text, start_date date, end_date date, room_number smallint, guest_id uuid, PRIMARY KEY (confirm_number, hotel_id, start_date, room_number)) "
query = query + "WITH comment ='Q6. Find reservations by confirmation number'"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS reservations_by_hotel_date"
query = query + "(hotel_id text, start_date date, end_date date, room_number smallint, confirm_number text, guest_id uuid, PRIMARY KEY ((hotel_id, start_date), room_number)) "
query = query + "WITH comment ='Q7. Find reservations by hotel and date'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "CREATE TABLE IF NOT EXISTS reservations_by_guest "
query = query + "(guest_last_name text, hotel_id text, room_number smallint, start_date date, end_date date, confirm_number text, guest_id uuid, PRIMARY KEY (guest_last_name, hotel_id)) "
query = query + "WITH comment ='Q8. Find reservations by guest name'"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    

    
query = "CREATE TABLE IF NOT EXISTS guests "
query = query + "(guest_id uuid, first_name text, last_name text, title text, emails set<text>, phone_numbers list<text>, address map<text, frozen<address>>, confirm_number text, PRIMARY KEY (guest_id))"
query = query + "WITH comment ='Q9. Find guest by ID'"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### If you need to drop table when you test, then below CQL can run

In [49]:
query = "drop table reservations_by_confirmation"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_guest"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table reservations_by_hotel_date"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table guests"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

In [50]:
session.shutdown()
cluster.shutdown()